<a href="https://colab.research.google.com/github/sorinNgit/Machine-Learning-Image-Classification/blob/main/Copy_of_Proiect_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip ai-unibuc-24-22-2021.zip


Streaming output truncated to the last 5000 lines.
  inflating: validation/030001.png   
  inflating: validation/030002.png   
  inflating: validation/030003.png   
  inflating: validation/030004.png   
  inflating: validation/030005.png   
  inflating: validation/030006.png   
  inflating: validation/030007.png   
  inflating: validation/030008.png   
  inflating: validation/030009.png   
  inflating: validation/030010.png   
  inflating: validation/030011.png   
  inflating: validation/030012.png   
  inflating: validation/030013.png   
  inflating: validation/030014.png   
  inflating: validation/030015.png   
  inflating: validation/030016.png   
  inflating: validation/030017.png   
  inflating: validation/030018.png   
  inflating: validation/030019.png   
  inflating: validation/030020.png   
  inflating: validation/030021.png   
  inflating: validation/030022.png   
  inflating: validation/030023.png   
  inflating: validation/030024.png   
  inflating: validation/030025.png   

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib.image import imread
import cv2 as cv
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score

out = open("output.txt", "w")

def normalize_img(path):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    normalizedImg = np.zeros((32, 32))
    normalizedImg = cv.normalize(img, normalizedImg, 0, 255, cv.NORM_MINMAX)

    #return normalizedImg.flatten()
    return normalizedImg

def load_data(data_dir, file_names):
    return [normalize_img(data_dir + "/" + name) for name in file_names]

def read_labels(path, column_names=['id', 'label']):
    return pd.read_csv(path, names=column_names)

def load_labeled_data(data_dir, df):
    data = load_data(data_dir, df.id)
    labels = list(df.label)
    return data, labels




DATA_DIR = ''
def load_training_dataset():
    print("Loading provided training dataset")
    train_df = read_labels(DATA_DIR + 'train.txt')
    return load_labeled_data(DATA_DIR + 'train', train_df)

def load_validation_dataset():
    print("Loading provided validation dataset")
    validation_df = read_labels(DATA_DIR + 'validation.txt')
    return load_labeled_data(DATA_DIR + 'validation', validation_df)

def load_labeled_dataset():
    train_data, train_labels = load_training_dataset()
    validation_data, validation_labels = load_validation_dataset()

    # Merge all labeled data into a single array
    data = np.stack(train_data + validation_data)
    labels = np.stack(train_labels + validation_labels)

    print("Loaded", len(data), "labeled samples")

    return data, labels

def load_test_dataset():
    print("Loading unlabeled dataset")

    test_df = read_labels(DATA_DIR + 'test.txt', column_names=['name'])

    test_df = test_df.set_index('name')

    return test_df, load_data(DATA_DIR + 'test', test_df.index)


train_data, train_labels = load_training_dataset()
validation_data, validation_labels = load_validation_dataset()
test_df,test_data = load_test_dataset()
#data, labels = load_labeled_dataset()
out.write("id,label\n")

#clf = svm.SVC(C=9, kernel='rbf')
#clf.fit(train_data, train_labels)
#validation_predict = clf.predict(validation_data)
#accuracy = accuracy_score(validation_labels, validation_predict)
#print(accuracy)

#for i,j in zip(test_df.index, test_predict):
#    out.write(str(i) + "," + str(j) + '\n')

#out.close()




Loading provided training dataset
Loading provided validation dataset
Loading unlabeled dataset


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization, SeparableConv2D
from keras.utils import normalize, to_categorical
from keras.losses import SparseCategoricalCrossentropy
train_normalized_data = np.array(train_data).reshape(len(train_data),32,32,1).astype('float32')
train_normalized_data /=255.0
train_normalized_data = np.array(train_labels)


X = np.array(train_data).reshape(len(train_data),32,32,1)
y = np.array(train_labels)
X = X.astype('float32')
X /= 255.0


Vimgs = np.array(validation_data).reshape(len(validation_data),32,32,1)
Vlbl = np.array(validation_labels)
Vimgs = Vimgs.astype('float32')
Vimgs /= 255.0

Timgs = np.array(test_data).reshape(len(test_data),32,32,1)
Timgs = Timgs.astype('float32')
Timgs /= 255.0


model = Sequential()

model.add(Conv2D(32,(3,3),activation = "relu", input_shape = (32,32,1), padding = 'valid'))
model.add(BatchNormalization(axis = -1))
model.add(SeparableConv2D(64, (3,3), activation = 'relu' , padding = 'valid'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(SeparableConv2D(64, (3,3), activation = 'relu' , padding = 'valid'))
model.add(BatchNormalization(axis = -1))
model.add(Conv2D(32,(3,3), activation = 'relu', padding = 'valid'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(9, activation = "softmax"))


model.compile(optimizer="adam", loss=SparseCategoricalCrossentropy(), metrics=["accuracy"])
#model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-3), loss=keras.losses.SparseCategoricalCrossentropy(), metrics='accuracy')

#train_datagen = ImageDataGenerator(
 #   rotation_angle = 45,
  #  width_shift_range = 0.2,
   # zoom_range = 0.2,
    #horizontal_flip = True)
#train_datagen.fit(train_data)

#train_generator = train_datagen.flow(
 #   train_data,
  #  train_labels,
   # batch_size = 32
#)

model.fit(X, y, epochs = 50, validation_data = (Vimgs, Vlbl))


#history = model.fit_generator(train_generator, epochs = 10, steps_per_epoch = 500, validation_data = (???),verbose = 1)

model.evaluate(Vimgs, Vlbl)

predicted_labels = model.predict_classes(Timgs)

out = open("output.txt", "w")
out.write("id,label\n")

for i,j in zip(test_df.index, predicted_labels):
    out.write(str(i) + "," + str(j) + '\n')


out.close()




Epoch 1/50
938/938 [==============================] - 8s 8ms/step - loss: 1.5225 - accuracy: 0.5376 - val_loss: 0.6713 - val_accuracy: 0.7782
Epoch 2/50
938/938 [==============================] - 7s 8ms/step - loss: 0.7866 - accuracy: 0.7175 - val_loss: 0.5931 - val_accuracy: 0.7938
Epoch 3/50
938/938 [==============================] - 7s 8ms/step - loss: 0.6783 - accuracy: 0.7536 - val_loss: 0.5242 - val_accuracy: 0.8220
Epoch 4/50
938/938 [==============================] - 7s 8ms/step - loss: 0.6180 - accuracy: 0.7773 - val_loss: 0.5109 - val_accuracy: 0.8206
Epoch 5/50
938/938 [==============================] - 7s 8ms/step - loss: 0.5720 - accuracy: 0.7929 - val_loss: 0.4808 - val_accuracy: 0.8284
Epoch 6/50
938/938 [==============================] - 8s 8ms/step - loss: 0.5442 - accuracy: 0.8047 - val_loss: 0.4360 - val_accuracy: 0.8430
Epoch 7/50
938/938 [==============================] - 7s 8ms/step - loss: 0.5124 - accuracy: 0.8174 - val_loss: 0.4481 - val_accuracy: 0.8422
Epoch 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
